In [53]:
!pip install xgboost

  Obtaining dependency information for xgboost from https://files.pythonhosted.org/packages/bc/43/242432efc3f60052a4a534dc4926b21e236ab4ec8d4920c593da3f65c65d/xgboost-2.0.2-py3-none-win_amd64.whl.metadata
   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.3/99.8 MB 10.6 MB/s eta 0:00:10
   ---------------------------------------- 0.7/99.8 MB 9.3 MB/s eta 0:00:11
   ---------------------------------------- 0.9/99.8 MB 7.2 MB/s eta 0:00:14
   ---------------------------------------- 1.1/99.8 MB 6.1 MB/s eta 0:00:17
   ---------------------------------------- 1.1/99.8 MB 6.1 MB/s eta 0:00:17
   ---------------------------------------- 1.1/99.8 MB 4.3 MB/s eta 0:00:24
    --------------------------------------- 1.4/99.8 MB 4.5 MB/s eta 0:00:23
    --------------------------------------- 1.6/99.8 MB 4.4 MB/s eta 0:00:23
    --------------------------------------- 1.8/99.8 MB 4.3 MB/s eta 0:00:23
    ---------------------------

In [54]:
import spectral
import matplotlib.pyplot as plt
import numpy as np
import spectral.io.aviris as aviris
import pandas as pd
import pysptools.eea as eea
import pysptools.noise as noise
import pandas as pd
import os

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,classification_report
from sklearn.neighbors import KNeighborsClassifier

from sklearn.ensemble import RandomForestClassifier

from xgboost import XGBClassifier

In [2]:
img = aviris.open('C:/Users/gabri/RFL0/f970619t01p02_r02_sc02.a.rfl', 'C:/Users/gabri/f970619t01p02_r02.a.spc')

In [3]:
img

	Data Source:   'C:/Users/gabri/RFL0/f970619t01p02_r02_sc02.a.rfl'
	# Rows:            512
	# Samples:         614
	# Bands:           224
	Interleave:        BIP
	Quantization:  16 bits
	Data format:     int16

# PURE PIXEL EXTRACTION

In [4]:
img_data=img.load()
    
np.set_printoptions(threshold=np.inf, linewidth=140)
    
img_2d=img_data.reshape((img_data.shape[0]*img_data.shape[1]), img_data.shape[2])

In [5]:
[purepixels,bands]=eea.eea.PPI(img_2d,10,10)

In [6]:
bands

array([111958, 104790, 104783, 104784, 104785, 104786, 104787, 104788, 104789, 104791], dtype=int64)

# WAVELENGTH EXTRACTION OF PPI

In [7]:
file_path = 'C:/Users/gabri/f970619t01p02_r02.a.spc'

In [8]:
column_names =['wavelength_center_position','fwhm_equivalent_gaussian','Uncertainty in wavelength center position',                             
                           'Uncertainty in FWHM for equivalent gaussian',                           
                           'channel_number']

In [9]:
df_wav_ppi = pd.read_csv(file_path, delimiter='\t', header=None, names=column_names)

In [10]:
wavelength=[]
wavelength = df_wav_ppi.wavelength_center_position 

In [11]:
len(wavelength)

224

In [12]:
wavelength[:10]

0    369.85
1    379.69
2    389.53
3    399.37
4    409.21
5    419.06
6    428.91
7    438.76
8    448.61
9    458.46
Name: wavelength_center_position, dtype: float64

In [13]:
max_ppi,min_ppi=max(wavelength),min(wavelength)

In [14]:
print(max_ppi)
print(min_ppi)

2506.81
369.85


In [15]:
ppi_wavelen=[]
for wave in wavelength:
    we=float(round(wave))
    ppi_wavelen.append(we)
wave_spec= pd.DataFrame({'wavelength':ppi_wavelen})

# PPI DATAFRAME

In [16]:
bands

array([111958, 104790, 104783, 104784, 104785, 104786, 104787, 104788, 104789, 104791], dtype=int64)

In [17]:
def read_and_process_file(file_path, wave_spec):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            if line.strip():  # Check if the line is not empty
                if "Name: " in line:
                    name_value = line.split("Name: ")[1].strip()
                elif "Class: " in line:
                    class_value = line.split("Class: ")[1].strip()
                elif len(line.split()) == 2 and all(value.replace('.', '').isdigit() for value in line.split()):
                    # Check if the line has two elements and both are numeric
                    data.append([float(value) for value in line.split()])

    df = pd.DataFrame(data, columns=['wavelength', f'reflectance#{class_value}#{name_value}'])
    df['wavelength'] = pd.to_numeric(df['wavelength'], errors='coerce') * 1000
    df[f'reflectance#{class_value}#{name_value}'] = pd.to_numeric(df[f'reflectance#{class_value}#{name_value}'], errors='coerce')
    return df[df['wavelength'].isin(wave_spec['wavelength']) == True]


In [18]:
def merge_files_into_dataframe(directory, wave_spec):
    dfs = pd.DataFrame({'wavelength': wave_spec['wavelength']})
    
    for filename in os.listdir(directory):
        if filename.endswith('.spectrum.txt'):
            file_path = os.path.join(directory, filename)
            df = read_and_process_file(file_path, wave_spec)
            dfs = pd.merge(dfs, df, on='wavelength', how='left')
    
    return dfs

In [19]:
directory = 'D:/data-20231104T081748Z-001/beckman/'
result_dataframe = merge_files_into_dataframe(directory, wave_spec)
print(result_dataframe)

C:\Users\gabri\AppData\Local\Temp\ipykernel_18492\3938279286.py:8: FutureWarning: Passing 'suffixes' which cause duplicate columns {'reflectance#Carbonates#Calcite CaCO_3_x'} in the result is deprecated and will raise a MergeError in a future version.
  dfs = pd.merge(dfs, df, on='wavelength', how='left')
C:\Users\gabri\AppData\Local\Temp\ipykernel_18492\3938279286.py:8: FutureWarning: Passing 'suffixes' which cause duplicate columns {'reflectance#Carbonates#Calcite CaCO_3_x'} in the result is deprecated and will raise a MergeError in a future version.
  dfs = pd.merge(dfs, df, on='wavelength', how='left')
C:\Users\gabri\AppData\Local\Temp\ipykernel_18492\3938279286.py:8: FutureWarning: Passing 'suffixes' which cause duplicate columns {'reflectance#Carbonates#Dolomite CaMg(CO_3)_2_x'} in the result is deprecated and will raise a MergeError in a future version.
  dfs = pd.merge(dfs, df, on='wavelength', how='left')
C:\Users\gabri\AppData\Local\Temp\ipykernel_18492\3938279286.py:8: Futur

     wavelength  reflectance#Arsenates#Mimetite Pb_5(AsO_4)_3Cl_x  \
0         370.0                                               NaN   
1         380.0                                               NaN   
2         390.0                                               NaN   
3         399.0                                               NaN   
4         409.0                                             13.05   
..          ...                                               ...   
219      2467.0                                               NaN   
220      2477.0                                               NaN   
221      2487.0                                               NaN   
222      2497.0                                               NaN   
223      2507.0                                               NaN   

     reflectance#Arsenates#Scorodite Fe^3+AsO_4.2H_2O_x  \
0                                                  NaN    
1                                                  Na

In [20]:
df_transposed = result_dataframe.transpose().reset_index()
df_transposed.set_axis(df_transposed.iloc[0], axis=1, inplace=True)
df_transposed = df_transposed[1:]
df_transposed[['value', 'class', 'sub - class']] = df_transposed['wavelength'].str.split("#",expand=True)

columns_to_drop = ['value', 'wavelength']
df_transposed = df_transposed.drop(columns=columns_to_drop)

filtered_columns = df_transposed.columns[df_transposed.notnull().all()]
filtered_df = df_transposed[filtered_columns]

col=list(filtered_df.columns)[:-2]

filtered_df['class'].value_counts()

C:\Users\gabri\AppData\Local\Temp\ipykernel_18492\1637771729.py:2: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df_transposed.set_axis(df_transposed.iloc[0], axis=1, inplace=True)


Silicates     216
Carbonates     45
Sulfates       39
Sulfides       34
Oxides         31
Borates        18
Halides        15
Phosphates     12
Arsenates       6
Elements        6
Hydroxides      5
Tungstates      3
Name: class, dtype: int64

In [21]:
filtered_df.head()

,409.0,419.0,429.0,439.0,449.0,458.0,468.0,478.0,488.0,498.0,...,2268.0,2288.0,2308.0,2328.0,2348.0,2368.0,2388.0,2408.0,class,sub - class
1,13.05,13.62,14.30,15.26,16.67,18.35,20.85,23.93,27.55,31.83,...,66.23,66.54,66.13,65.83,66.28,66.43,66.15,65.96,Arsenates,Mimetite Pb_5(AsO_4)_3Cl_x
2,7.78,7.58,7.22,8.47,10.38,11.01,11.53,12.57,14.02,15.78,...,8.89,8.56,7.54,7.07,7.22,7.06,6.63,6.11,Arsenates,Scorodite Fe^3+AsO_4.2H_2O_x
3,60.13,61.51,63.37,65.49,67.67,69.63,72.14,74.54,76.63,78.82,...,91.49,91.59,91.53,91.73,91.81,91.84,91.69,91.65,Arsenates,Mimetite Pb_5(AsO_4)_3Cl_y
4,29.67,30.59,30.79,36.28,41.59,43.15,44.75,47.49,51.50,54.92,...,55.51,54.90,53.02,52.09,52.37,51.98,50.92,49.54,Arsenates,Scorodite Fe^3+AsO_4.2H_2O_y
5,27.81,29.23,31.25,33.68,36.52,39.37,42.97,47.06,51.34,55.70,...,80.62,80.80,80.45,80.51,80.91,80.94,80.83,80.70,Arsenates,Mimetite Pb_5(AsO_4)_3Cl


In [22]:
row_indices_to_print = list(bands) # list of ppi bands

specific_rows = img_2d[row_indices_to_print,:]

df_pppi = pd.DataFrame(specific_rows.T, columns=row_indices_to_print)

df_pppi.insert(0, 'wavelength', ppi_wavelen) #inserting the purepixel wavelengths

selected_df2 = df_pppi[df_pppi['wavelength'].isin(col)]

selecteddf_t=selected_df2.T
selecteddf_t.set_axis(selecteddf_t.iloc[0], axis=1, inplace=True)
selecteddf_t= selecteddf_t[1:]

selecteddf_t

C:\Users\gabri\AppData\Local\Temp\ipykernel_18492\2060080328.py:12: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  selecteddf_t.set_axis(selecteddf_t.iloc[0], axis=1, inplace=True)


wavelength,409.0,419.0,429.0,439.0,449.0,458.0,468.0,478.0,488.0,498.0,...,1928.0,2248.0,2268.0,2288.0,2308.0,2328.0,2348.0,2368.0,2388.0,2408.0
111958,0.2846,0.3114,0.3287,0.3425,0.3538,0.3585,0.3612,0.3657,0.3714,0.3800,...,0.0,0.5891,0.5879,0.5866,0.5802,0.5858,0.5857,0.5723,0.5696,0.5482
104790,0.1111,0.1195,0.1285,0.1331,0.1378,0.1430,0.1459,0.1479,0.1511,0.1547,...,0.0,0.2904,0.2893,0.2855,0.2763,0.2644,0.2611,0.2608,0.2608,0.2410
104783,0.1111,0.1218,0.1337,0.1380,0.1483,0.1521,0.1535,0.1562,0.1595,0.1638,...,0.0,0.2510,0.2534,0.2539,0.2460,0.2399,0.2372,0.2341,0.2297,0.2189
104784,0.1111,0.1231,0.1327,0.1422,0.1502,0.1543,0.1571,0.1603,0.1636,0.1689,...,0.0,0.2566,0.2524,0.2568,0.2447,0.2398,0.2313,0.2302,0.2251,0.2206
104785,0.1127,0.1231,0.1327,0.1396,0.1502,0.1543,0.1556,0.1590,0.1636,0.1689,...,0.0,0.2702,0.2648,0.2655,0.2560,0.2470,0.2405,0.2420,0.2387,0.2288
104786,0.1111,0.1231,0.1327,0.1422,0.1496,0.1532,0.1550,0.1575,0.1622,0.1664,...,0.0,0.2662,0.2698,0.2667,0.2625,0.2529,0.2405,0.2403,0.2439,0.2268
104787,0.1046,0.1148,0.1244,0.1305,0.1391,0.1441,0.1485,0.1501,0.1538,0.1580,...,0.0,0.2596,0.2583,0.2527,0.2460,0.2413,0.2375,0.2343,0.2352,0.2295
104788,0.0966,0.1053,0.1119,0.1216,0.1299,0.1340,0.1372,0.1395,0.1435,0.1480,...,0.0,0.2778,0.2757,0.2744,0.2650,0.2615,0.2525,0.2551,0.2511,0.2358
104789,0.0966,0.1018,0.1098,0.1165,0.1214,0.1267,0.1276,0.1317,0.1343,0.1388,...,0.0,0.2930,0.2871,0.2814,0.2663,0.2644,0.2612,0.2556,0.2456,0.2333
104791,0.1225,0.1302,0.1410,0.1488,0.1554,0.1605,0.1632,0.1650,0.1689,0.1735,...,0.0,0.2930,0.2921,0.2901,0.2817,0.2682,0.2668,0.2624,0.2585,0.2484


# ML Model

In [26]:
X = filtered_df.drop(['class','sub - class'], axis=1)
y = filtered_df['class']

In [27]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [43]:
#set(filtered_df['class'])
# Use Counter to get the frequency of each element
from collections import Counter
frequency_counter = Counter(filtered_df['class'])

# Print unique values and their frequencies
print("Unique values and their frequencies:")
for value, frequency in frequency_counter.items():
    print(f"{value}: {frequency}")

Unique values and their frequencies:
Arsenates: 6
Borates: 18
Carbonates: 45
Elements: 6
Halides: 15
Hydroxides: 5
Oxides: 31
Phosphates: 12
Silicates: 216
Sulfates: 39
Sulfides: 34
Tungstates: 3


In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [29]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [30]:
svm_model = SVC(kernel='linear', C=1.0, random_state=42)
svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_test)

In [31]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 60.47%


In [32]:

# Standardize the new features using the same scaler used for training data
X_new = scaler.transform(selecteddf_t)

# Make predictions on the new data
y_pred_new = svm_model.predict(X_new)

# If you used LabelEncoder for the target variable, you might want to inverse_transform the predictions
# to get back the original text labels
y_pred_labels = label_encoder.inverse_transform(y_pred_new)

# Display or use the predictions as needed
print("Predictions on new data:")
print(y_pred_labels)

Predictions on new data:
['Sulfides' 'Sulfides' 'Sulfides' 'Sulfides' 'Sulfides' 'Sulfides' 'Sulfides' 'Sulfides' 'Sulfides' 'Sulfides']


In [44]:
knn_model = KNeighborsClassifier(n_neighbors=12)  # You can adjust the number of neighbors

# Train the KNN model
knn_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = knn_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')



Accuracy: 46.51%


In [45]:
# Standardize the new features using the same scaler used for training data
X_new = scaler.transform(selecteddf_t)

# Make predictions on the new data
y_pred_new = knn_model.predict(X_new)

y_pred_labels = label_encoder.inverse_transform(y_pred_new)

# Display or use the predictions as needed
print("Predictions on new data:")
print(y_pred_labels)


Predictions on new data:
['Oxides' 'Oxides' 'Oxides' 'Oxides' 'Oxides' 'Oxides' 'Oxides' 'Oxides' 'Oxides' 'Oxides']


In [49]:
# Create a Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier on the training data
rf_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_classifier.predict(X_test)

# Evaluate the performance of the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Display a classification report
print('Classification Report:')
print(classification_report(y_test, y_pred))

Accuracy: 0.59
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.25      0.25      0.25         4
           2       0.25      0.11      0.15         9
           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00         3
           5       0.00      0.00      0.00         1
           6       0.71      0.83      0.77         6
           7       0.00      0.00      0.00         2
           8       0.72      0.88      0.79        43
           9       0.33      0.25      0.29         8
          10       0.57      0.57      0.57         7
          11       0.00      0.00      0.00         1

    accuracy                           0.59        86
   macro avg       0.24      0.24      0.24        86
weighted avg       0.52      0.59      0.55        86



D:\anoconda\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anoconda\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anoconda\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [57]:
X_new = scaler.transform(selecteddf_t)

new_predictions = rf_classifier.predict(X_new)

y_pred_labels = label_encoder.inverse_transform(new_predictions)

# Display the predictions
print('Predictions for the new data:')
print(y_pred_labels)

Predictions for the new data:
['Sulfides' 'Sulfides' 'Sulfides' 'Sulfides' 'Sulfides' 'Sulfides' 'Sulfides' 'Sulfides' 'Sulfides' 'Sulfides']


In [55]:
xgb_classifier = XGBClassifier(objective='multi:softmax', num_class=len(set(y)), random_state=42)

# Train the classifier on the training data
xgb_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = xgb_classifier.predict(X_test)

# Evaluate the performance of the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')



Accuracy: 0.56


In [56]:
# Display a classification report
print('Classification Report:')
print(classification_report(y_test, y_pred))




Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.00      0.00      0.00         4
           2       0.17      0.11      0.13         9
           3       1.00      1.00      1.00         1
           4       0.00      0.00      0.00         3
           5       0.00      0.00      0.00         1
           6       0.44      0.67      0.53         6
           7       0.00      0.00      0.00         2
           8       0.76      0.81      0.79        43
           9       0.40      0.25      0.31         8
          10       0.45      0.71      0.56         7
          11       0.00      0.00      0.00         1

    accuracy                           0.56        86
   macro avg       0.27      0.30      0.28        86
weighted avg       0.51      0.56      0.53        86



D:\anoconda\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anoconda\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anoconda\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [58]:
X_new = scaler.transform(selecteddf_t)

xgb_new_predictions = xgb_classifier.predict(X_new)

y_pred_labels = label_encoder.inverse_transform(xgb_new_predictions)

# Display the predictions
print('XGBoost Predictions for the new data:')
print(y_pred_labels)

XGBoost Predictions for the new data:
['Sulfides' 'Sulfides' 'Sulfides' 'Sulfides' 'Sulfides' 'Sulfides' 'Sulfides' 'Sulfides' 'Sulfides' 'Sulfides']
